<a href="https://colab.research.google.com/github/lawrennd/qig-code/blob/main/examples/boring_game_dynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The "Boring" Game: Dynamics from the LME Origin

### Neil D. Lawrence

### December 2025


In [ ]:
# Auto-install QIG package if not available
import os

try:
    import qig
except ImportError:
    print("📦 Installing QIG package...")
    %pip install -q git+https://github.com/lawrennd/qig-code.git
    import qig
    print("✓ QIG package installed!")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qig import partial_trace, von_neumann_entropy, create_lme_state, marginal_entropies
from qig.pair_operators import bell_state_density_matrix, product_of_bell_states
from qig import partial_trace, von_neumann_entropy, create_lme_state, marginal_entropies
from qig.pair_operators import bell_state_density_matrix, product_of_bell_states


In [ ]:
# Plot configuration
plt.style.use('seaborn-v0_8-whitegrid' if 'seaborn-v0_8-whitegrid' in plt.style.available else 'default')
big_wide_figsize = (10, 5)
big_figsize = (8, 6)
plt.rcParams.update({
    'font.size': 14,
    'font.family': 'serif',
    'axes.labelsize': 18,
    'axes.titlesize': 16,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 12,
})

# Create output directory
os.makedirs('./diagrams', exist_ok=True)
print("✓ Configuration complete")

## Summary

When the inaccessible game starts from the Locally Maximally Entangled (LME) origin—a product of Bell states—the marginal entropy constraint is **automatically satisfied** along the entire gradient flow. The game is "boring" in the sense that the constrained and unconstrained dynamics are identical: the constraint projection $\Pi_\parallel = \mathbf{I}$ throughout.

This document explains why this happens and when the game becomes non-trivial.


## Setup

### The Bell State

For a pair of qutrits ($d=3$), the Bell state is
$$
|\Phi\rangle = \frac{1}{\sqrt{3}}(|00\rangle + |11\rangle + |22\rangle)
$$
This is a **maximally entangled** state with:
- Joint entropy: $H = 0$ (pure state)
- Marginal entropies: $h_A = h_B = \log 3$ (marginals are $\tfrac{\mathbf{I}}{3}$)
- Mutual information: $I(A:B) = 2 \log 3$ (maximum)

### The Constraint

The marginal entropy constraint is
$$
C = \sum_i h_i = h_A + h_B
$$
For the Bell state: $C = 2 \log 3$.

### The Target State

The maximum entropy state is $\tfrac{\mathbf{I}}{9}$ (maximally mixed), with:
- Joint entropy: $H = \log 9$
- Marginal entropies: $h_A = h_B = \log 3$ (marginals are *also* $\tfrac{\mathbf{I}}{3}$!)
- Mutual information: $I(A:B) = 0$



## Key Observation: Both Endpoints Have the Same Marginals


In [ ]:
d = 3
D = d * d  # 9

# Bell state using qig API
rho_bell = bell_state_density_matrix(d)

# Maximally mixed state
I_D = np.eye(D) / D

# Compute marginals using qig API
rho_A_bell = partial_trace(rho_bell, [d, d], keep=0)
rho_A_mixed = partial_trace(I_D, [d, d], keep=0)

print("Marginal of Bell state:")
print(np.round(rho_A_bell.real, 4))

print("\nMarginal of I/9:")
print(np.round(rho_A_mixed.real, 4))

print(f"\nBoth are I/3: {np.allclose(rho_A_bell, np.eye(d)/d) and np.allclose(rho_A_mixed, np.eye(d)/d)}")


In [ ]:
# ✓ Refactored: Now using qig.pair_operators.bell_state_density_matrix() and qig.partial_trace()
print(f"✓ Using qig API: bell_state_density_matrix(d={d})")
print(f"✓ Using qig API: partial_trace(rho, [d, d], keep=0)")

**Why?**
- Bell state: $\rho_A = \text{tr}_B(|\Phi\rangle\langle\Phi|) = \tfrac{\mathbf{I}}{3}$ (defining property of maximal entanglement)
- $\tfrac{\mathbf{I}}{9} = \tfrac{(\mathbf{I}\otimes\mathbf{I})}{9}$: $\rho_A = \text{tr}_B\left(\tfrac{\mathbf{I}\otimes\mathbf{I}}{9}\right) = \mathbf{I}\cdot\tfrac{\text{tr}(\mathbf{I})}{9} = \mathbf{I}\cdot \tfrac{3}{9} = \tfrac{\mathbf{I}}{3}$

## The Gradient Flow Preserves C Exactly

The steepest entropy ascent dynamics naturally preserve the constraint.


In [ ]:
import numpy as np

In [ ]:
d = 3
D = 9

# Bell state using qig API
rho_bell = bell_state_density_matrix(d)

# Constraint function using qig API
def constraint_C(rho, d):
    """Sum of marginal entropies using qig.marginal_entropies"""
    return np.sum(marginal_entropies(rho, [d, d]))

# Regularize to avoid pure state singularity
eps = 0.01
rho = (1-eps) * rho_bell + eps * np.eye(D)/D

# Gradient flow simulation
dt = 0.001
n_steps = 300
C_initial = constraint_C(rho, d)

print(f"{'Step':<8} {'H':<12} {'C':<12} {'dC':<14}")
print("-" * 46)

for step in range(n_steps + 1):
    H = von_neumann_entropy(rho)  # Using qig API
    C = constraint_C(rho, d)

    if step % 50 == 0:
        print(f"{step:<8} {H:<12.6f} {C:<12.6f} {C - C_initial:<+14.10f}")

    if step < n_steps:
        # Entropy gradient
        eigvals, eigvecs = np.linalg.eigh(rho)
        eigvals = np.maximum(eigvals, 1e-15)
        log_rho = eigvecs @ np.diag(np.log(eigvals)) @ eigvecs.T.conj()

        grad = -log_rho - np.eye(D)
        grad = grad - np.trace(grad) * np.eye(D) / D  # Trace-preserving

        # Update and project to valid density matrix
        rho_new = rho + dt * grad
        rho_new = (rho_new + rho_new.T.conj()) / 2
        eigvals_new, eigvecs_new = np.linalg.eigh(rho_new)
        eigvals_new = np.maximum(eigvals_new, 0)
        eigvals_new = eigvals_new / np.sum(eigvals_new)
        rho = eigvecs_new @ np.diag(eigvals_new) @ eigvecs_new.T.conj()

In [ ]:
# ✓ Refactored: Now using qig.von_neumann_entropy() and qig.marginal_entropies()
print("✓ Using qig API: von_neumann_entropy(rho) for joint entropy")
print("✓ Using qig API: marginal_entropies(rho, [d, d]) for constraint C")

The constraint $C = 2 \log 3$ is preserved to machine precision at every step!

## Multiple Pairs: Same Result

For $n$ pairs, the product of Bell states $|\Phi_1\rangle\otimes|\Phi_2\rangle\otimes\cdots\otimes|\Phi_n\rangle$ evolves toward $\mathbf{I}/D$ where $D = 9^n$. Both have all marginals $= \mathbf{I}/3$.


In [ ]:
import numpy as np

In [ ]:
d = 3
n_pairs = 2
D = d ** (2 * n_pairs)  # 81
dims = [d, d, d, d]  # A1, B1, A2, B2

# Product of Bell states using qig API
psi_product = product_of_bell_states(n_pairs=2, d=3)
rho_product = np.outer(psi_product, psi_product.conj())

# Constraint using qig API
def total_C(rho):
    """Sum of all marginal entropies using qig.marginal_entropies"""
    return np.sum(marginal_entropies(rho, dims))

# Regularize with whole-system regularization
eps = 0.01
rho = (1-eps) * rho_product + eps * np.eye(D)/D

# Gradient flow
dt = 0.0002
n_steps = 150
C_initial = total_C(rho)

print(f"2 PAIRS: Bell⊗Bell -> I/81")
print(f"{'Step':<8} {'H':<12} {'C':<12} {'dC':<14}")
print("-" * 46)

for step in range(n_steps + 1):
    H = von_neumann_entropy(rho)  # Using qig API
    C = total_C(rho)

    if step % 30 == 0:
        print(f"{step:<8} {H:<12.6f} {C:<12.6f} {C - C_initial:<+14.10f}")

    if step < n_steps:
        eigvals, eigvecs = np.linalg.eigh(rho)
        eigvals = np.maximum(eigvals, 1e-15)
        log_rho = eigvecs @ np.diag(np.log(eigvals)) @ eigvecs.T.conj()

        grad = -log_rho - np.eye(D)
        grad = grad - np.trace(grad) * np.eye(D) / D

        rho_new = rho + dt * grad
        rho_new = (rho_new + rho_new.T.conj()) / 2
        eigvals_new, eigvecs_new = np.linalg.eigh(rho_new)
        eigvals_new = np.maximum(eigvals_new, 0)
        eigvals_new = eigvals_new / np.sum(eigvals_new)
        rho = eigvecs_new @ np.diag(eigvals_new) @ eigvecs_new.T.conj()

print(f"\nTarget: H = log 81 = {np.log(81):.6f}, C = 4 log 3 = {4*np.log(3):.6f}")

In [ ]:
# ✓ Refactored: Now using qig.pair_operators.product_of_bell_states() and qig.marginal_entropies()
print("✓ Using qig API: product_of_bell_states(n_pairs=2, d=3)")
print("✓ Using qig API: marginal_entropies(rho, dims) for constraint")

## Intuition: What's Happening to the Entanglement?

The Bell state density matrix has a specific structure:


In [ ]:
import numpy as np

In [ ]:
d = 3
# Bell state using qig API
rho_bell = bell_state_density_matrix(d)

print("Bell state rho:")
print("Rows/cols: |00>, |01>, |02>, |10>, |11>, |12>, |20>, |21>, |22>")
print(np.round(rho_bell.real, 3))


**Two components**
- **Diagonal** (positions $0,4,8$): The classical correlation—only $|00\rangle$, $|11\rangle$, $|22\rangle$ are occupied
- **Off-diagonal** ($|00\rangle\langle 11|$, $|00\rangle\langle 22|$, $|11\rangle\langle 22|$): The quantum coherence = entanglement

**During the gradient flow**
1. Off-diagonal coherences *decay* $\rightarrow$ entanglement destroyed
2. Diagonal *spreads* $\rightarrow$ all 9 states become equally likely ($\tfrac{1}{9}$)
3. Marginals stay $\tfrac{\mathbf{I}}{3}$ throughout


In [ ]:
import numpy as np

In [ ]:
d = 3
D = 9

# Bell state using qig API
rho_bell = bell_state_density_matrix(d)

# Mutual information using qig API: I(A:B) = H(A) + H(B) - H(A,B)
def mutual_information(rho, d):
    """Compute mutual information using qig API"""
    h_marginals = marginal_entropies(rho, [d, d])
    H_AB = von_neumann_entropy(rho)
    return np.sum(h_marginals) - H_AB

eps = 0.01
rho = (1-eps) * rho_bell + eps * np.eye(D)/D

dt = 0.001
n_steps = 400

print(f"{'Step':<6} {'H':<10} {'I(A:B)':<10} {'Off-diag':<12} {'h_A':<10}")
print("-" * 50)

for step in range(n_steps + 1):
    H = von_neumann_entropy(rho)  # Using qig API
    I_AB = mutual_information(rho, d)
    h_A = marginal_entropies(rho, [d, d])[0]  # Using qig API

    # Off-diagonal magnitude
    off_diag = np.sqrt(sum(np.abs(rho[i,j])**2 for i in range(D) for j in range(D) if i != j))

    if step % 80 == 0:
        print(f"{step:<6} {H:<10.4f} {I_AB:<10.4f} {off_diag:<12.6f} {h_A:<10.4f}")

    if step < n_steps:
        eigvals, eigvecs = np.linalg.eigh(rho)
        eigvals = np.maximum(eigvals, 1e-15)
        log_rho = eigvecs @ np.diag(np.log(eigvals)) @ eigvecs.T.conj()
        grad = -log_rho - np.eye(D)
        grad = grad - np.trace(grad) * np.eye(D) / D
        rho_new = rho + dt * grad
        rho_new = (rho_new + rho_new.T.conj()) / 2
        eigvals_new, eigvecs_new = np.linalg.eigh(rho_new)
        eigvals_new = np.maximum(eigvals_new, 0)
        eigvals_new = eigvals_new / np.sum(eigvals_new)
        rho = eigvecs_new @ np.diag(eigvals_new) @ eigvecs_new.T.conj()


In [ ]:
# ✓ Refactored: Now using qig API throughout
print("✓ Using qig API: von_neumann_entropy(rho) for joint entropy")
print("✓ Using qig API: marginal_entropies(rho, [d, d]) for h_A, h_B")
print("✓ Mutual information I(A:B) = sum(h) - H computed via qig")

| Quantity | Changes | Direction |
|----------|---------|-----------|
| $H$ (joint entropy) | ✓ | $0 \to \log 9$ |
| $I(A:B)$ (mutual information) | ✓ | $2 \log 3 \to 0$ |
| Off-diagonal coherences | ✓ | Decay to 0 |
| $h_A = h_B$ (marginal entropies) | ✗ | Constant at $\log 3$ |
| $C = h_A + h_B$ | ✗ | Constant at $2 \log 3$ |

## When Does the Game Become Interesting?

The constraint only engages when starting from states where marginals $\neq \mathbf{I}/d$:


In [ ]:
import numpy as np



In [ ]:
d = 3
n_pairs = 2
D = 81
dims = [d, d, d, d]

# Bell state (pair 1) using qig API
rho_bell = bell_state_density_matrix(d)

# |00><00| (pure product state for pair 2)
rho_00 = np.zeros((9, 9))
rho_00[0, 0] = 1  # |00><00|

# Mixed marginals: Bell (h=log3) x |00> (h=0)
rho_mixed = np.kron(rho_bell, rho_00)

# Constraint using qig API
def total_C(rho):
    """Sum of all marginal entropies using qig.marginal_entropies"""
    return np.sum(marginal_entropies(rho, dims))

print("State: Bell ⊗ |00><00|")
h_mixed = marginal_entropies(rho_mixed, dims)  # Using qig API
print(f"Marginal entropies: h_A1={h_mixed[0]:.4f}, "
      f"h_B1={h_mixed[1]:.4f}, "
      f"h_A2={h_mixed[2]:.4f}, "
      f"h_B2={h_mixed[3]:.4f}")
print(f"C = {total_C(rho_mixed):.4f}")
print(f"Target (I/81): C = {total_C(np.eye(D)/D):.4f}")

# Regularize and take one step
eps = 0.01
rho = (1-eps) * rho_mixed + eps * np.eye(D)/D

eigvals, eigvecs = np.linalg.eigh(rho)
log_rho = eigvecs @ np.diag(np.log(eigvals)) @ eigvecs.T.conj()
grad = -log_rho - np.eye(D)
grad = grad - np.trace(grad) * np.eye(D) / D

rho_new = rho + 0.001 * grad
rho_new = (rho_new + rho_new.T.conj()) / 2

print(f"\nAfter one unconstrained step:")
print(f"C_before = {total_C(rho):.6f}")
print(f"C_after  = {total_C(rho_new):.6f}")
print(f"dC = {total_C(rho_new) - total_C(rho):.6f}")
print("\n=> Constraint would ENGAGE to project out this change!")


## Summary

| Initial State | $C$ | Target $C$ | Constraint Active? |
|---------------|-----|------------|-------------------|
| $|\Phi\rangle \otimes |\Phi\rangle \otimes \cdots$ | $n \times 2 \log d$ | $n \times 2 \log d$ | **No** |
| Mixed marginals | $< n \times 2 \log d$ | $n \times 2 \log d$ | **Yes** |

The "boring" game from the LME origin:
1. All marginals are $\tfrac{\mathbf{I}}{d}$, and stay $\tfrac{\mathbf{I}}{d}$ along the flow
2. The constraint is automatically satisfied
3. Constrained = unconstrained dynamics
4. Entanglement dissipates while marginal structure is preserved

The game becomes interesting when:
1. Different subsystems have different marginal structures
2. The unconstrained flow would change $C$
3. The constraint projection actively shapes the dynamics
